In [0]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive

fuse: mountpoint is not empty
fuse: if you are sure this is safe, use the 'nonempty' mount option


In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [1]:
cd gdrive/My\ Drive/DMIS/Re-Implementation/Transformer_Re-Implementation

/content/gdrive/My Drive/DMIS/Re-Implementation/Transformer_Re-Implementation


In [2]:
ls

config.py                  model.py                    temp.py
dataset/                   preprocess_blueprint.ipynb  train_blueprint2.ipynb
Embedding_blueprint.ipynb  process.py                  train_blueprint.ipynb
embedding.py               __pycache__/                train_colab.ipynb
model_blueprint.ipynb      README.md                   train.py


## ================== drive mounting ======================
## Train

In [0]:
import model as trf
from process import get_data
from train import *
from config import *

In [0]:
import torch
import torch.nn as nn
from torchtext import data, datasets

In [0]:
data_path = "dataset/"
file_name = "train."

In [0]:
print_every = 50
epochs = 10

In [0]:
SRC, TGT, trn = get_data(data_path + file_name)

In [0]:
pad_idx = TGT.vocab.stoi["<pad>"]

In [0]:
model = trf.Transformer(len(SRC.vocab), len(TGT.vocab), N = 6)
#model.cuda()
criterion = nn.CrossEntropyLoss(ignore_index = 1)
#criterion.cuda()
model_opt = torch.optim.Adam(model.parameters(), lr = .1, betas=(.9, .98), eps = 1e-9)

In [0]:
train_iter = data.Iterator(trn, batch_size = BATCH_SIZE,
                          device = DEVICE_SET,
                          repeat = False)

In [13]:
train_model(1, model, criterion, model_opt, train_iter)

Epoch Step: 1 Loss: 0.100652 per Sec: 4.889545
Epoch Step: 101 Loss: 0.103308 per Sec: 364.470698
Epoch Step: 201 Loss: 0.083216 per Sec: 380.054562
Epoch Step: 301 Loss: 0.108804 per Sec: 411.327192


KeyboardInterrupt: ignored

In [0]:
model.save(model.state_dict(), "model_save")